# Chapter 3.2: Context Providers & Advanced RAG Patterns

In this notebook, you'll learn how the Agent Framework's **Context Providers** abstract away the complexity of RAG while providing powerful capabilities for multi-turn conversations and multiple knowledge sources.

---

## Learning Objectives

By the end of this notebook, you will:

1. Understand the Context Provider lifecycle (`invoking()` and `invoked()`)
2. Use the built-in `AzureAISearchContextProvider` for automatic RAG
3. Combine multiple knowledge sources with `AggregateContextProvider`
4. Build custom Context Providers for specialized retrieval logic

---

## Prerequisites

Before starting this notebook:
1. Complete `03.1-rag-fundamentals.ipynb` to create your search index
2. Have your `.env` file configured with search credentials

In [31]:
# Environment Setup & Validation
import sys
sys.path.insert(0, '..')

from workshop_utils import validate_env
validate_env()

✅ Setup validated! All required environment variables are set.


True

In [32]:
import os
from dotenv import load_dotenv

load_dotenv()

# Azure AI Search configuration
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("SEARCH_API_KEY")
index_name = 'test-index-123'

# Azure OpenAI configuration (same as other notebooks)
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")

print(f"Search Endpoint: {search_endpoint}")
print(f"Index Name: {index_name}")
print(f"OpenAI Endpoint: {openai_endpoint}")

Search Endpoint: https://maf-workshop-ai-search.search.windows.net
Index Name: test-index-123
OpenAI Endpoint: https://aoai-sweden-gbb-dev.openai.azure.com/


---

## Part 1: Understanding Context Providers

**Context Providers** are middleware components in the Agent Framework that run before and after every agent invocation. They solve all the problems we identified with manual RAG:

```
┌────────────────────────────────────────────────────────────────────┐
│                    Context Provider Pipeline                       │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│  User Message ──► invoking() ──► Agent ──► invoked() ──► Response │
│                      │                         │                   │
│                      │                         │                   │
│                      ▼                         ▼                   │
│              Inject Context             Extract/Store Info         │
│              (from search)              (for future use)           │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

### Key Methods

| Method | When It Runs | Purpose |
|--------|--------------|----------|
| `invoking()` | Before LLM call | Inject retrieved context, instructions, or tools |
| `invoked()` | After LLM call | Extract information, update memory, log interactions |

### Why Context Providers?

| Manual RAG Problem | Context Provider Solution |
|-------------------|---------------------------|
| Boilerplate code | Declarative configuration |
| No multi-turn support | Built-in with `AgentThread` |
| Manual token management | Configurable `top_k` |
| Complex multi-source | `AggregateContextProvider` |
| State serialization | Built-in `serialize()` |

---

## Part 2: AzureAISearchContextProvider

The Agent Framework provides a built-in `AzureAISearchContextProvider` that automatically:

1. Searches your index on every user message
2. Injects relevant documents as context
3. Handles authentication and connection management

In [33]:
from openai import AsyncAzureOpenAI
from agent_framework import ChatAgent
from agent_framework.azure import AzureOpenAIChatClient, AzureAISearchContextProvider


async def get_embedding(text: str) -> list[float]:
    """Generate embedding for search queries."""
    async with AsyncAzureOpenAI(
        api_version=api_version,
        azure_endpoint=openai_endpoint,
        api_key=openai_api_key,
    ) as client:
        response = await client.embeddings.create(
            input=text,
            model=embedding_deployment
        )
        return response.data[0].embedding


# Create chat client (AzureOpenAIChatClient doesn't require explicit cleanup)
chat_client = AzureOpenAIChatClient(
    endpoint=openai_endpoint,
    api_key=openai_api_key,
    api_version=api_version,
    deployment_name=chat_deployment,
)

# Create the context provider with embedding function for vector search
search_provider = AzureAISearchContextProvider(
    endpoint=search_endpoint,
    index_name=index_name,
    api_key=search_key,
    mode="semantic",
    top_k=3,
    semantic_configuration_name=f"{index_name}-semantic-configuration",
    vector_field_name="text_vector",
    embedding_function=get_embedding,
)

async with (
    search_provider,
    ChatAgent(
        chat_client=chat_client,
        name="HRAgent",
        instructions="""
        You are a helpful HR assistant for Contoso Electronics employees.
        
        Answer questions using ONLY the context provided to you.
        If the context doesn't contain the answer, say "I don't have that information."
        Be concise and cite specific plan names when applicable.
        """,
        context_provider=search_provider,
    ) as agent,
):
    response = await agent.run("What is the deductible for Northwind Health Plus?")
    print(response.text)


k_nearest_neighbors is not a known attribute of class <class 'azure.search.documents._generated.models._models_py3.VectorizedQuery'> and will be ignored


The deductible for Northwind Health Plus is $1,500 per year for individuals and $3,000 per year for families for in-network services. The deductible is reset each year on the plan's renewal date. There is no calendar year deductible for out-of-network services, but higher cost sharing applies. Certain services, like preventive care and emergency services, are exempt from the calendar year deductible.


### Compare: Manual RAG vs. Context Provider

| Aspect | Manual RAG | Context Provider |
|--------|------------|------------------|
| **Code Lines** | ~30 lines per agent | ~5 lines configuration |
| **Search Timing** | Manual before each call | Automatic on every invocation |
| **Multi-turn** | Custom thread management | Built-in with `AgentThread` |
| **Multiple Sources** | Custom orchestration | Just add more providers |
| **Serialization** | Custom code | Built-in `serialize()` |

### Streaming Responses

Context Providers work seamlessly with streaming:

In [34]:
async with (
    search_provider,
    ChatAgent(
        chat_client=chat_client,
        name="HRAgent",
        instructions="You are a helpful HR assistant. Answer based on provided context.",
        context_provider=search_provider,
    ) as agent,
):
    print("Agent: ", end="", flush=True)
    
    async for chunk in agent.run_stream("What is covered under preventive care?"):
        if chunk.text:
            print(chunk.text, end="", flush=True)
    
    print()

Agent: 

k_nearest_neighbors is not a known attribute of class <class 'azure.search.documents._generated.models._models_py3.VectorizedQuery'> and will be ignored


Based on the provided context, the following are **covered under preventive care** for both Northwind Standard and Northwind Health Plus plans:

**Northwind Standard:**
- Physicals and vaccinations
- Health screenings and tests (e.g., blood pressure, cholesterol, diabetes)
- Counseling (lifestyle and nutrition counseling)
- Immunizations
- Vision and hearing screenings
- Other preventive services recommended by the U.S. Preventive Services Task Force

**Northwind Health Plus:**
- Routine physicals (with no cost-sharing)
- Vaccinations (e.g., flu, shingles, measles, mumps, rubella)
- Screenings (e.g., cancer, diabetes, high blood pressure)

**General Notes and Exceptions:**
- Preventive care services are available at no extra cost when you are enrolled in these plans.
- Cosmetic services and any service not medically necessary are not covered.
- Only services recommended by your doctor or the plan are covered.
- Over-the-counter medications and non-FDA-approved drugs are not covered unl

### Multi-turn Conversations with AgentThread

Context Providers maintain state across conversation turns using `AgentThread`:

In [24]:
async with (
    search_provider,
    ChatAgent(
        chat_client=chat_client,
        name="HRAgent",
        instructions="You are a helpful HR assistant. Answer based on provided context. Be concise.",
        context_provider=search_provider,
    ) as agent,
):
    # Create a conversation thread
    thread = agent.get_new_thread()
    
    # Multi-turn conversation
    questions = [
        "What is the deductible for Northwind Health Plus?",
        "And what about the coinsurance?",  # Follow-up referring to same plan
        "How does that compare to Northwind Standard?",  # Comparison
    ]
    
    for question in questions:
        print(f"\nUser: {question}")
        response = await agent.run(question, thread=thread)
        print(f"Agent: {response.text}")


User: What is the deductible for Northwind Health Plus?


k_nearest_neighbors is not a known attribute of class <class 'azure.search.documents._generated.models._models_py3.VectorizedQuery'> and will be ignored


Agent: The deductible for Northwind Health Plus is $1,500 per year for individuals and $3,000 per year for families for in-network services. There is no calendar year deductible for out-of-network services, but higher cost sharing applies. The deductible resets each year on the plan’s renewal date. Certain services, such as preventive care and emergency services, are exempt from the deductible.

User: And what about the coinsurance?


k_nearest_neighbors is not a known attribute of class <class 'azure.search.documents._generated.models._models_py3.VectorizedQuery'> and will be ignored


Agent: For Northwind Health Plus, after you meet your deductible, you are responsible for coinsurance—a percentage of the cost for covered services. The standard coinsurance rate is typically 20% for most services. This means that for a service with an allowed amount of $100, you would pay $20 and the plan would pay $80. 

Preventive care services are exempt from coinsurance (you pay nothing for these). Coinsurance rates may vary for certain services; check your plan documents for specifics. Out-of-network services usually have higher cost sharing.

User: How does that compare to Northwind Standard?


k_nearest_neighbors is not a known attribute of class <class 'azure.search.documents._generated.models._models_py3.VectorizedQuery'> and will be ignored


Agent: Compared to Northwind Health Plus, Northwind Standard may offer a less comprehensive network of in-network providers, so it’s important to research whether the available providers fit your needs. Regarding prescription drug coverage, Northwind Standard does offer it, but you should not assume that all drugs will be low or no cost; review the plan details to see which prescriptions are covered and what your out-of-pocket costs may be.

Additionally, Northwind Standard follows rules for primary and secondary coverage: if you have another health plan (like through a spouse), Northwind Standard typically acts as secondary coverage, paying eligible expenses left after the primary plan has paid. Coverage details, such as deductibles and cost sharing, may differ from Northwind Health Plus, so compare the specifics to see which plan best fits your situation.


---

## Part 3: Multiple Knowledge Sources with AggregateContextProvider

When you need to combine multiple knowledge sources, use the `AggregateContextProvider`. This pattern aggregates context from all sources before injecting it into the agent.

> **Note**: The `ChatAgent` accepts a single `context_provider` parameter. To use multiple sources, we wrap them in an `AggregateContextProvider`.

### Exercise 1: Create a Second Knowledge Base

Before using multiple sources, create a second index with the documents in `data/index2/`:

- Employee benefits overview
- Company policies and handbook  
- Wellness programs (PerksPlus)
- Role descriptions

**Option A: Use the portal wizard** (faster for learning)
1. Upload documents from `data/index2/` to a new blob container
2. Create a new index using the Import Data wizard
3. Add `INDEX2_NAME=<your-index-name>` to your `.env` file

**Option B: Programmatic creation** (reuse code from 03.1)
1. Modify the indexing code to point to `data/index2/`
2. Use a different index name (e.g., `hr-general-index`)

In [26]:
# Check if second index is configured
load_dotenv(override=True)
index2_name = 'test-index-2'

if index2_name:
    print(f"✅ Second index configured: {index2_name}")
else:
    print("⚠️ INDEX2_NAME not found in .env - complete Exercise 1 first")
    print("   You can still proceed with a single index, but the multi-source examples won't work.")

✅ Second index configured: test-index-2


### AggregateContextProvider Implementation

The `AggregateContextProvider` combines multiple context providers by:
1. Calling `invoking()` on all providers in parallel
2. Merging their returned contexts (instructions, messages, tools)
3. Calling `invoked()` on all providers after the agent responds

In [27]:
import asyncio
from collections.abc import MutableSequence, Sequence
from contextlib import AsyncExitStack
from types import TracebackType
from typing import Any

from agent_framework import ContextProvider, Context, ChatMessage


class AggregateContextProvider(ContextProvider):
    """
    A ContextProvider that combines multiple context providers.
    
    It delegates events to all providers and aggregates their responses.
    This allows you to combine multiple knowledge sources into a single provider.
    """
    
    def __init__(self, providers: list[ContextProvider]):
        self.providers = providers
        self._exit_stack: AsyncExitStack | None = None
    
    async def invoking(
        self,
        messages: ChatMessage | MutableSequence[ChatMessage],
        **kwargs: Any
    ) -> Context:
        """Aggregate context from all providers."""
        # Call all providers in parallel
        contexts = await asyncio.gather(*[
            provider.invoking(messages, **kwargs) 
            for provider in self.providers
        ])
        
        # Merge all contexts
        instructions = ""
        all_messages: list[ChatMessage] = []
        all_tools: list = []
        
        for ctx in contexts:
            if ctx.instructions:
                instructions += ctx.instructions + "\n"
            if ctx.messages:
                all_messages.extend(ctx.messages)
            if ctx.tools:
                all_tools.extend(ctx.tools)
        
        return Context(
            instructions=instructions.strip() if instructions else None,
            messages=all_messages if all_messages else None,
            tools=all_tools if all_tools else None,
        )
    
    async def invoked(
        self,
        request_messages: ChatMessage | Sequence[ChatMessage],
        response_messages: ChatMessage | Sequence[ChatMessage] | None = None,
        invoke_exception: Exception | None = None,
        **kwargs: Any,
    ) -> None:
        """Notify all providers after invocation."""
        await asyncio.gather(*[
            provider.invoked(
                request_messages=request_messages,
                response_messages=response_messages,
                invoke_exception=invoke_exception,
                **kwargs,
            )
            for provider in self.providers
        ])
    
    async def __aenter__(self):
        """Enter all provider contexts."""
        self._exit_stack = AsyncExitStack()
        await self._exit_stack.__aenter__()
        
        for provider in self.providers:
            await self._exit_stack.enter_async_context(provider)
        
        return self
    
    async def __aexit__(
        self,
        exc_type: type[BaseException] | None,
        exc_val: BaseException | None,
        exc_tb: TracebackType | None,
    ) -> None:
        """Exit all provider contexts."""
        if self._exit_stack:
            await self._exit_stack.__aexit__(exc_type, exc_val, exc_tb)
            self._exit_stack = None


print("AggregateContextProvider defined ✅")

AggregateContextProvider defined ✅


In [ ]:
# Skip this cell if INDEX2_NAME is not configured
if not index2_name:
    print("Skipping multi-source example - INDEX2_NAME not configured")
else:
    # Create individual providers for each knowledge source
    health_plans_provider = AzureAISearchContextProvider(
        endpoint=search_endpoint,
        index_name=index_name,
        api_key=search_key,
        mode="semantic",
        top_k=3,
        semantic_configuration_name=f"{index_name}-semantic-configuration",
        vector_field_name="text_vector",
        embedding_function=get_embedding,
    )

    general_hr_provider = AzureAISearchContextProvider(
        endpoint=search_endpoint,
        index_name=index2_name,
        api_key=search_key,
        mode="semantic",
        top_k=3,
        semantic_configuration_name=f"{index2_name}-semantic-configuration",
        vector_field_name="text_vector",
        embedding_function=get_embedding,
    )

    # Combine providers
    composite_provider = AggregateContextProvider([
        health_plans_provider,
        general_hr_provider,
    ])

    # Test questions that span both knowledge bases
    TEST_QUESTIONS = [
        "What's the PerksPlus program?",          # General HR (index2)
        "What is the deductible for Northwind Standard?",  # Health plans (index1)
        "What benefits are available to employees?",       # Spans both
    ]

    async with (
        composite_provider,
        ChatAgent(
            chat_client=chat_client,
            name="ComprehensiveHRAgent",
            instructions="""
            You are a helpful HR assistant for Contoso Electronics employees.

            You have access to two knowledge bases:
            1. **Health Plan Details**: Northwind Health Plus and Standard plan specifics
            2. **General HR Knowledge**: Company policies, benefits overview, PerksPlus

            Guidelines:
            - Answer using ONLY the context provided
            - Keep responses to 2-3 sentences
            - Cite specific sources when relevant
            """,
            context_provider=composite_provider,
        ) as agent,
    ):
        for question in TEST_QUESTIONS:
            print(f"\nUser: {question}")
            print("Agent: ", end="", flush=True)
            
            async for chunk in agent.run_stream(question):
                if chunk.text:
                    print(chunk.text, end="", flush=True)
            
            print()


---

## Part 4: Custom Context Providers

You can build custom Context Providers for specialized retrieval or context injection logic. Common use cases:

- **User profile injection** - Add user preferences to every request
- **Time-aware context** - Inject current date/time information
- **Custom retrieval** - Integrate with non-Azure search systems
- **Memory extraction** - Store facts from conversations for future use

In [29]:
from datetime import datetime


class TimeAwareContextProvider(ContextProvider):
    """
    A context provider that injects the current date and time.
    
    Useful for agents that need to understand temporal context,
    like deadlines, enrollment periods, or time-sensitive policies.
    """
    
    async def invoking(
        self,
        messages: ChatMessage | MutableSequence[ChatMessage],
        **kwargs: Any
    ) -> Context:
        current_time = datetime.now().strftime("%A, %B %d, %Y at %I:%M %p")
        return Context(
            instructions=f"Current date and time: {current_time}"
        )


class UserProfileContextProvider(ContextProvider):
    """
    A context provider that injects user profile information.
    
    This allows personalized responses based on the user's
    enrollment status, plan selection, or employment details.
    """
    
    def __init__(self, user_profile: dict):
        self.user_profile = user_profile
    
    async def invoking(
        self,
        messages: ChatMessage | MutableSequence[ChatMessage],
        **kwargs: Any
    ) -> Context:
        profile_text = "\n".join([
            f"- {key}: {value}" 
            for key, value in self.user_profile.items()
        ])
        return Context(
            instructions=f"User Profile:\n{profile_text}"
        )


print("Custom context providers defined ✅")

Custom context providers defined ✅


In [ ]:
# Combine custom providers with search provider
user_profile = {
    "Name": "Alice Johnson",
    "Department": "Engineering",
    "Current Plan": "Northwind Health Plus",
    "Employment Status": "Full-time",
}

time_provider = TimeAwareContextProvider()
profile_provider = UserProfileContextProvider(user_profile)

# Create a search provider for knowledge retrieval
knowledge_provider = AzureAISearchContextProvider(
    endpoint=search_endpoint,
    index_name=index_name,
    api_key=search_key,
    mode="semantic",
    top_k=3,
    semantic_configuration_name=f"{index_name}-semantic-configuration",
    vector_field_name="text_vector",
    embedding_function=get_embedding,
)

# Combine all providers
combined_provider = AggregateContextProvider([
    time_provider,
    profile_provider,
    knowledge_provider,
])

async with (
    combined_provider,
    ChatAgent(
        chat_client=chat_client,
        name="PersonalizedHRAgent",
        instructions="""
        You are a personalized HR assistant.
        
        Use the user profile to personalize your responses.
        Reference their current plan when answering health questions.
        Be aware of the current date for any time-sensitive information.
        """,
        context_provider=combined_provider,
    ) as agent,
):
    # The agent now has access to time, user profile, AND search results
    response = await agent.run("What is my deductible?")
    print(f"Agent: {response.text}")


### Exercise 2: Build a Memory Extraction Provider

Create a custom context provider that:
1. In `invoked()`, extracts any facts the user mentions (like preferences)
2. In `invoking()`, injects those stored facts as context

This creates a simple long-term memory for the agent.

<details>
<summary>Click to see solution</summary>

```python
class SimpleMemoryProvider(ContextProvider):
    """A context provider that extracts and remembers user facts."""
    
    def __init__(self):
        self.memories: list[str] = []
    
    async def invoking(
        self,
        messages: ChatMessage | MutableSequence[ChatMessage],
        **kwargs: Any
    ) -> Context:
        if not self.memories:
            return Context()
        
        memory_text = "\n".join([f"- {m}" for m in self.memories])
        return Context(
            instructions=f"Remembered facts about the user:\n{memory_text}"
        )
    
    async def invoked(
        self,
        request_messages: ChatMessage | Sequence[ChatMessage],
        response_messages: ChatMessage | Sequence[ChatMessage] | None = None,
        **kwargs: Any,
    ) -> None:
        # Simple extraction: look for "I prefer" or "I like" statements
        msgs = [request_messages] if isinstance(request_messages, ChatMessage) else list(request_messages)
        
        for msg in msgs:
            content = str(msg.content).lower()
            if "i prefer" in content or "i like" in content:
                self.memories.append(str(msg.content))
```

</details>

---

## Summary

In this notebook, you learned:

1. **Context Provider Lifecycle** - `invoking()` runs before the LLM, `invoked()` runs after
2. **AzureAISearchContextProvider** - Built-in RAG with automatic search and context injection
3. **AggregateContextProvider** - Combine multiple knowledge sources elegantly
4. **Custom Providers** - Build specialized context injection for your use cases

### Key Takeaways

- Context Providers abstract away RAG complexity while remaining flexible
- Use `context_provider` (singular) parameter - wrap multiple providers in `AggregateContextProvider`
- Multi-turn conversations work automatically with `AgentThread`
- Custom providers enable personalization, memory, and specialized retrieval

### Advanced Topics (Not Covered)

The Agent Framework supports additional patterns:

- **Agentic mode** - Multi-hop reasoning with Azure AI Search Knowledge Bases
- **TextSearchProvider** - Function-based RAG with custom search delegates
- **Mem0 Provider** - Long-term memory across conversation sessions
- **Redis Provider** - Fast retrieval with full-text and vector search

See: https://github.com/microsoft/agent-framework/tree/main/python/samples/getting_started/context_providers